저용량

In [ ]:
import cv2
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image
from math import sqrt,exp

import copy
img = cv2.imread("과제2_저용량.png") #원본이미지
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #bgr을 rgb로 바꾸는 코드
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
plt.imshow(img_gray, cmap = 'gray')

In [ ]:
#픽셀값의 확률분포 계산

def histo(img_gray):
    histogram = np.zeros(256)
    h_input,w_input = img_gray.shape
    #w_input = 512
    #h_input = 512

    for r in range(h_input):
        for c in range(w_input):
            histogram[img_gray[r,c]] += 1

    histogram = histogram/(w_input*h_input)

    # print(histogram)
    return histogram


def img_pro(img_gray):
    img_proba = np.zeros(256)
    hist = histo(img_gray)
    img_proba[0] = hist[0]
    for k in range(1,256):
        img_proba[k] = img_proba[k-1] + hist[k]

    # print(img_proba)
    return img_proba

In [ ]:
prob = img_pro(img_gray)
plt.plot(prob)

In [ ]:
def compress(img_gray):
    n = 4
    
    flatten_img = img_gray.flatten()
    proba = img_pro(img_gray)
    comp = np.zeros(int(len(flatten_img)/n), dtype=np.float32)

    for i in range(len(comp)):
        high =1
        low=0
        data = flatten_img[i*n:(i+1)*n]
        for encoding_i in range(n):
            data_range = high - low
            high = low + data_range * proba[data[encoding_i]]
            if(data[encoding_i] == 0):
                low = low
            else:
                low = low + data_range * proba[data[encoding_i]-1]
        comp[i] = low+(high-low)/2
    return comp

In [ ]:
#def asasas(data, codebook) 함수로 만들어라

def decompress(data_enc, codebook, img_gray):
    h_input,w_input = img_gray.shape
    proba = img_pro(img_gray)
    data_dec = np.zeros(codebook)
    recov = np.zeros(len(data_enc)*codebook, dtype=np.float32)

    for i in range(len(data_enc)):
        high =1
        low=0      
        for decoding_i in range(codebook):
            data_range = high - low
            c=(data_enc[i] - low) / data_range
            for k in range(256):
                if c < proba[k]:
                    break

            data_dec[decoding_i] = k

            high = low + data_range * proba[k]
            if(k == 0):
                low = len
            else:
                low = low + data_range * proba[k-1]
        recov[i*codebook:(i+1)*codebook] = data_dec
    recov = recov.reshape(-1,w_input)
    return recov


In [ ]:
comp = compress(img_gray)
print(comp.shape)

In [ ]:
np.save('압축이미지1', comp)
np.save('원본이미지1', img_gray.astype('float32'))

In [ ]:
rec = decompress(comp, 4, img_gray)
plt.imshow(rec, cmap='gray')

고용량

In [ ]:
import cv2
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image
from math import sqrt,exp

import copy
img = cv2.imread("과제2.png") #원본이미지
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #bgr을 rgb로 바꾸는 코드
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
plt.imshow(img_gray, cmap = 'gray')

In [ ]:
#픽셀값의 확률분포 계산

def histo(img_gray):
    histogram = np.zeros(256)
    h_input,w_input = img_gray.shape
    #w_input = 512
    #h_input = 512

    for r in range(h_input):
        for c in range(w_input):
            histogram[img_gray[r,c]] += 1

    histogram = histogram/(w_input*h_input)

    # print(histogram)
    return histogram


def img_pro(img_gray):
    img_proba = np.zeros(256)
    hist = histo(img_gray)
    img_proba[0] = hist[0]
    for k in range(1,256):
        img_proba[k] = img_proba[k-1] + hist[k]

    # print(img_proba)
    return img_proba

In [ ]:
def compress(img_gray):
    n = 4
    
    flatten_img = img_gray.flatten()
    proba = img_pro(img_gray)
    comp = np.zeros(int(len(flatten_img)/n), dtype=np.float32)

    for i in range(len(comp)):
        high =1
        low=0
        data = flatten_img[i*n:(i+1)*n]
        for encoding_i in range(n):
            data_range = high - low
            high = low + data_range * proba[data[encoding_i]]
            if(data[encoding_i] == 0):
                low = low
            else:
                low = low + data_range * proba[data[encoding_i]-1]
        comp[i] = low+(high-low)/2
    return comp

In [ ]:
#def asasas(data, codebook) 함수로 만들어라

def decompress(data_enc, codebook, img_gray):
    h_input,w_input = img_gray.shape
    proba = img_pro(img_gray)
    data_dec = np.zeros(codebook)
    recov = np.zeros(len(data_enc)*codebook, dtype=np.float32)

    for i in range(len(data_enc)):
        high =1
        low=0      
        for decoding_i in range(codebook):
            data_range = high - low
            c=(data_enc[i] - low) / data_range
            for k in range(256):
                if c < proba[k]:
                    break

            data_dec[decoding_i] = k

            high = low + data_range * proba[k]
            if(k == 0):
                low = len
            else:
                low = low + data_range * proba[k-1]
        recov[i*codebook:(i+1)*codebook] = data_dec
    recov = recov.reshape(-1,w_input)
    return recov


In [ ]:
comp = compress(img_gray)
print(comp.shape)

In [ ]:
np.save('압축이미지2', comp)
np.save('원본이미지2', img_gray.astype('float32'))

In [ ]:
rec = decompress(comp, 4, img_gray)
plt.imshow(rec, cmap='gray')

In [ ]:
# data = img_gray[100,:]
# #print(data)

# data_cha = np.zeros(w_input,dtype=int)
# data_cha[0] = data[0]
# for i in range(1,w_input):
#     data_cha[i] = int(data[i]) - int(data[i-1])

# print(data_cha)